In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from numba import jit, prange, njit
from blimpy import Waterfall
import time
import random
import datetime
from scipy import spatial
from numpy import dot
from numpy.linalg import norm
import math
from sklearn.metrics import silhouette_score
import sys
sys.path.insert(1, '../ML_Training')
sys.path.insert(2, '../GBT_pipeline')
from preprocess_dynamic import get_data
from single_search import search_model_eval, combine
from skimage.transform import rescale, resize, downscale_local_mean
import gc
from keras.regularizers import l1, l2
from data_generation_multicore import create_data_set_multicore
from multiprocessing import Pool
import functools
from sys import getsizeof

/home/pma/.local/lib/python3.6/site-packages/numba/core/decorators.py:255: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)
/home/pma/.local/lib/python3.6/site-packages/numba/core/decorators.py:255: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)


In [2]:
NUM_SAMPLES=39
data, false_data, true_data = create_data_set_multicore(NUM_SAMPLES=NUM_SAMPLES, snr_base=20, snr_range = 10, factor=1)

Creating True
39
(39, 6, 16, 2048)
(39, 6, 16, 2048)
(39, 6, 16, 2048) (39, 6, 16, 256)
(39, 6, 16, 256)
(234, 16, 256, 1)
Creating False
(234, 6, 16, 2048)
(234, 6, 16, 2048) (234, 6, 16, 256)
(234, 6, 16, 256)
Creating True
(117, 6, 16, 2048)
(117, 6, 16, 2048) (117, 6, 16, 256)
(117, 6, 16, 256)
(117, 6, 16, 2048)
(117, 6, 16, 2048) (117, 6, 16, 256)
(117, 6, 16, 256)
(234, 6, 16, 256, 1)


In [3]:
for i in range(6):
    plt.figure(figsize=(10,4))
    plt.xlabel("Fchans")
    plt.ylabel("Time")
    plt.imshow(true_data[0,i,:,:], interpolation='nearest', cmap=plt.get_cmap('hot'), aspect=2)
    plt.show()

In [4]:
NUM_SAMPLES = 39
data_test, false_data_test, true_data_test = create_data_set_multicore(NUM_SAMPLES=NUM_SAMPLES, snr_base=20, snr_range = 10, factor=1)

Creating True
39
(39, 6, 16, 2048)
(39, 6, 16, 2048)
(39, 6, 16, 2048) (39, 6, 16, 256)
(39, 6, 16, 256)
(234, 16, 256, 1)
Creating False
(234, 6, 16, 2048)
(234, 6, 16, 2048) (234, 6, 16, 256)
(234, 6, 16, 256)
Creating True
(117, 6, 16, 2048)
(117, 6, 16, 2048) (117, 6, 16, 256)
(117, 6, 16, 256)
(117, 6, 16, 2048)
(117, 6, 16, 2048) (117, 6, 16, 256)
(117, 6, 16, 256)
(234, 6, 16, 256, 1)


In [5]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder,alpha=10, beta=2, gamma = 10, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        self.true_loss_tracker = keras.metrics.Mean(name="true_loss")
        self.false_loss_tracker = keras.metrics.Mean(name="false_loss")
        
        self.true_score_tracker = keras.metrics.Mean(name="true_score")
        self.false_score_tracker = keras.metrics.Mean(name="false_score")
        
        
        self.total_loss_tracker_validation = keras.metrics.Mean(name="val_total_loss")
        self.reconstruction_loss_tracker_validation = keras.metrics.Mean(
            name="val_reconstruction_loss"
        )
        self.kl_loss_tracker_validation = keras.metrics.Mean(name="val_kl_loss")
        self.false_loss_tracker_validation = keras.metrics.Mean(name="val_false_loss")
        self.true_loss_tracker_validation = keras.metrics.Mean(name="val_true_loss")
        
        self.true_score_tracker_validation = keras.metrics.Mean(name="true_score")
        self.false_score_tracker_validation = keras.metrics.Mean(name="false_score")
        
        
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.count=1

    @tf.function
    def loss_diff(self, a,b):
        return 1/ self.loss_same(a,b)
   
    @tf.function
    def loss_same(self, a,b):
        return tf.math.reduce_mean(tf.math.reduce_euclidean_norm(a-b, axis=1))

    
    @tf.function
    def true_score(self, true_data,training=True):
        similarity = 0
        difference = 0
        same = 0
        a1 = self.encoder(true_data[:,0,:,:,:], training)[2]
        b = self.encoder(true_data[:,1,:,:,:], training)[2]
        a2 = self.encoder(true_data[:,2,:,:,:],training)[2]
        c = self.encoder(true_data[:,3,:,:,:], training)[2]
        a3 = self.encoder(true_data[:,4,:,:,:], training)[2]
        d = self.encoder(true_data[:,5,:,:,:], training)[2]

        same+= self.loss_same(a1,a2)
        same+= self.loss_same(a1,a3)
        
        same+= self.loss_same(a2,a1)
        same+= self.loss_same(a2,a3)
        
        same+= self.loss_same(a3,a2)
        same+= self.loss_same(a3,a1)
        
        same+= self.loss_same(b,c)
        same+= self.loss_same(b,d)
        
        same+= self.loss_same(c,b)
        same+= self.loss_same(c,d)
        
        same+= self.loss_same(d,b)
        same+= self.loss_same(d,c)
        
        difference += self.loss_same(a1,b)
        difference += self.loss_same(a1,c)
        difference += self.loss_same(a1,d)
        
        difference += self.loss_same(a2,b)
        difference += self.loss_same(a2,c)
        difference += self.loss_same(a2,d)
        
        difference += self.loss_same(a3,b)
        difference += self.loss_same(a3,c)
        difference += self.loss_same(a3,d)
        
        if same > difference:
            similarity += (difference - same)/same
        else:
            similarity += (difference - same)/difference
 
        return 10*(1-similarity)/(2*(similarity+1))
    
    @tf.function
    def false_score(self, true_data,training=True):
        similarity = 0
        difference = 0
        same = 0
        a1 = self.encoder(true_data[:,0,:,:,:], training)[2]
        b = self.encoder(true_data[:,1,:,:,:], training)[2]
        a2 = self.encoder(true_data[:,2,:,:,:],training)[2]
        c = self.encoder(true_data[:,3,:,:,:], training)[2]
        a3 = self.encoder(true_data[:,4,:,:,:], training)[2]
        d = self.encoder(true_data[:,5,:,:,:], training)[2]

        same+= self.loss_same(a1,a2)
        same+= self.loss_same(a1,a3)
        
        same+= self.loss_same(a2,a1)
        same+= self.loss_same(a2,a3)
        
        same+= self.loss_same(a3,a2)
        same+= self.loss_same(a3,a1)
        
        same+= self.loss_same(b,c)
        same+= self.loss_same(b,d)
        
        same+= self.loss_same(c,b)
        same+= self.loss_same(c,d)
        
        same+= self.loss_same(d,b)
        same+= self.loss_same(d,c)
        
        difference += self.loss_same(a1,b)
        difference += self.loss_same(a1,c)
        difference += self.loss_same(a1,d)
        
        difference += self.loss_same(a2,b)
        difference += self.loss_same(a2,c)
        difference += self.loss_same(a2,d)
        
        difference += self.loss_same(a3,b)
        difference += self.loss_same(a3,c)
        difference += self.loss_same(a3,d)
        
        if same > difference:
            similarity += -(difference - same)/same
        else:
            similarity += -(difference - same)/difference
        
        return 10*(1-similarity)/(2*(similarity+1))
    
    @tf.function
    def true_clustering(self, true_data,training=True):
        similarity = 0
        difference = 0
        same = 0
        a1 = self.encoder(true_data[:,0,:,:,:], training)[2]
        b = self.encoder(true_data[:,1,:,:,:], training)[2]
        a2 = self.encoder(true_data[:,2,:,:,:],training)[2]
        c = self.encoder(true_data[:,3,:,:,:], training)[2]
        a3 = self.encoder(true_data[:,4,:,:,:], training)[2]
        d = self.encoder(true_data[:,5,:,:,:], training)[2]

        difference += self.loss_diff(a1,b)
        difference += self.loss_diff(a1,c)
        difference += self.loss_diff(a1,d)

        difference += self.loss_diff(a2,b)
        difference += self.loss_diff(a2,c)
        difference += self.loss_diff(a2,d)

        difference += self.loss_diff(a3,b)
        difference += self.loss_diff(a3,c)
        difference += self.loss_diff(a3,d)

        same += self.loss_same(a1,a2)
        same += self.loss_same(a1,a3)
        same += self.loss_same(a2,a3)
        
        same += self.loss_same(b,c)
        same += self.loss_same(c,d)
        same += self.loss_same(b,d)
        

        similarity += same+difference
        return similarity
    
    @tf.function
    def false_clustering(self, false_data, training=True):

        similarity = 0
        difference = 0
        same = 0
        a1 = self.encoder(false_data[:,0,:,:,:], training)[2]
        b = self.encoder(false_data[:,1,:,:,:], training)[2]
        a2 = self.encoder(false_data[:,2,:,:,:],training)[2]
        c = self.encoder(false_data[:,3,:,:,:], training)[2]
        a3 = self.encoder(false_data[:,4,:,:,:], training)[2]
        d = self.encoder(false_data[:,5,:,:,:], training)[2]

        difference += self.loss_same(a1,b)
        difference += self.loss_same(a1,c)
        difference += self.loss_same(a1,d)

        difference += self.loss_same(a2,b)
        difference += self.loss_same(a2,c)
        difference += self.loss_same(a2,d)

        difference += self.loss_same(a3,b)
        difference += self.loss_same(a3,c)
        difference += self.loss_same(a3,d)

        same += self.loss_same(a1,a2)
        same += self.loss_same(a1,a3)
        same += self.loss_same(a2,a3)
        
        same += self.loss_same(b,c)
        same += self.loss_same(c,d)
        same += self.loss_same(b,d)
        
        similarity += same+difference
        return similarity
    
    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]
    
    def train_step(self, data):
        self.count+=1
        cluster_loss =0
        x, y = data
        true_data = x[1]
        false_data = x[2]
        x= x[0]
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(x)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(y, reconstruction), axis=(1, 2)
                )
            )/(16*256)
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))

            false_loss = self.false_clustering(false_data)
            true_loss = self.true_clustering(true_data)
            
#             false_score = self.false_score(false_data)
#             true_score = self.true_score(true_data)
            false_score =0
            true_score = 0
            
            total_loss = reconstruction_loss/(16*256) + self.beta*kl_loss +self.alpha*(1*true_loss+false_loss)+self.gamma*(false_score+true_score)
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        self.false_loss_tracker.update_state(false_loss)
        self.true_loss_tracker.update_state(true_loss)
        
        self.false_score_tracker.update_state(false_score)
        self.true_score_tracker.update_state(true_score)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
            "false_loss": self.false_loss_tracker.result(),
            "true_loss": self.true_loss_tracker.result(),
            "false_score": self.false_score_tracker.result(),
            "true_score": self.true_score_tracker.result()
        }
    

    def test_step(self, data):
        # Unpack the data
        x, y = data
        true_data = x[1]
        false_data = x[2]
        x= x[0]
        z_mean, z_log_var, z = self.encoder(x, training=False)
        reconstruction = self.decoder(z, training=False)
        reconstruction_loss = tf.reduce_mean(
            tf.reduce_sum(
                keras.losses.binary_crossentropy(y, reconstruction), axis=(1, 2)
            )
        )/(16*256)
        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
        false_loss = self.false_clustering(false_data, False)
        true_loss = self.true_clustering(true_data, False)
#         false_score = self.false_score(false_data)
#         true_score = self.true_score(true_data)
        false_score = 0
        true_score =0
        total_loss = reconstruction_loss + self.beta*kl_loss +self.alpha*(1*true_loss+false_loss)+self.gamma*(false_score+true_score)
        
        
        self.total_loss_tracker_validation.update_state(total_loss)
        self.reconstruction_loss_tracker_validation.update_state(reconstruction_loss)
        self.kl_loss_tracker_validation.update_state(kl_loss)
        self.false_loss_tracker_validation.update_state(false_loss)
        self.true_loss_tracker_validation.update_state(true_loss)
        
        self.false_score_tracker_validation.update_state(false_score)
        self.true_score_tracker_validation.update_state(true_score)
        
        return {
            "loss": self.total_loss_tracker_validation.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker_validation.result(),
            "kl_loss": self.kl_loss_tracker_validation.result(),
            "false_loss": self.false_loss_tracker_validation.result(),
            "true_loss": self.true_loss_tracker_validation.result(),
            "false_score": self.false_score_tracker_validation.result(),
            "true_score": self.true_score_tracker_validation.result()
        }

In [6]:
from tensorflow.python.client import device_lib
gpus = tf.config.experimental.list_physical_devices('GPU')

In [7]:
class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [8]:
def sample_creation(inputs):
    z_mean = inputs[0]
    z_log_var = inputs[1]
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [9]:
def build_model(latent_dim=8,dens_lay=512, kernel=(3,3), alpha=10, beta=2, gamma=10):
    
    encoder_inputs = keras.Input(shape=(16, 256, 1))
    x = layers.Conv2D(16, kernel, activation="relu", strides=2, padding="same")(encoder_inputs)
    x = layers.Conv2D(16, kernel, activation="relu", strides=1, padding="same")(x)
    x = layers.Conv2D(32, kernel, activation="relu", strides=2, padding="same")(x)
    x = layers.Conv2D(32, kernel, activation="relu", strides=1, padding="same")(x)
    x = layers.Conv2D(32, kernel, activation="relu", strides=1, padding="same")(x)
    x = layers.Conv2D(64, kernel, activation="relu", strides=2,padding="same")(x)
    x = layers.Conv2D(64, kernel, activation="relu", strides=1, padding="same")(x)
    x = layers.Conv2D(128, kernel, activation="relu", strides=1, padding="same")(x)
    x = layers.Conv2D(128, kernel, activation="relu", strides=2, padding="same")(x)
    x = layers.Flatten()(x)
#     x = layers.Dropout(0.5, activity_regularizer=l1(0.001))(x)
    x = layers.Dense(dens_lay, activation="relu", activity_regularizer=l1(0.001),
                     kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(x)
#     x = layers.Dropout(0.5, activity_regularizer=l1(0.001))(x)
    z_mean = layers.Dense(latent_dim, name="z_mean", activity_regularizer=l1(0.001),
                          kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(x)
    z_log_var = layers.Dense(latent_dim, name="z_log_var" , activity_regularizer=l1(0.001),
                             kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(x)
    z = Sampling()([z_mean, z_log_var])
    encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
    encoder.summary()
    
    latent_inputs = keras.Input(shape=(latent_dim,))
    x = layers.Dense(dens_lay, activation="relu", activity_regularizer=l1(0.001), 
                     kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(latent_inputs)
#     x = layers.Dropout(0.5)(x)
    x = layers.Dense(1* 16 * 128, activation="relu", activity_regularizer=l1(0.001),
                     kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(x)
#     x = layers.Dropout(0.5)(x)
    x = layers.Reshape((1,16, 128))(x)
    x = layers.Conv2DTranspose(128, kernel, activation="relu", strides=2,padding="same")(x)
    x = layers.Conv2DTranspose(128, kernel, activation="relu", strides=1, padding="same")(x)
    x = layers.Conv2DTranspose(64, kernel, activation="relu", strides=1, padding="same")(x)
    x = layers.Conv2DTranspose(64, kernel, activation="relu", strides=2, padding="same")(x)
    x = layers.Conv2DTranspose(32, kernel, activation="relu", strides=1, padding="same")(x)
    x = layers.Conv2DTranspose(32, kernel, activation="relu", strides=1, padding="same")(x)
    x = layers.Conv2DTranspose(32, kernel, activation="relu", strides=2, padding="same")(x)
    x = layers.Conv2DTranspose(16, kernel, activation="relu", strides=1, padding="same")(x)
    x = layers.Conv2DTranspose(16, kernel, activation="relu", strides=2, padding="same")(x)
    decoder_outputs = layers.Conv2DTranspose(1, kernel, activation="sigmoid", padding="same")(x)
    decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
    decoder.summary()

    vae = VAE(encoder, decoder, alpha, beta, gamma)
    vae.compile(optimizer=keras.optimizers.Adam(lr=0.005))
    return vae

strategy = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.ReductionToOneDevice())

alpha, beta, gamma = 1,1,0
latent_dim=8
dens_lay=512
kernel=(3,3)


model_num = 46
name = "VAE-BLPC0-ENCODER_compressed_256v%d"%(model_num)
print(name)

print("Model " +name+": \(\\alpha : %3d\),  \(\\beta : %3d\),  \(\gamma : %3d\), latent_dim:%3d, dense:%3d " % (alpha, beta, gamma, latent_dim, dens_lay))

with strategy.scope():
    model = build_model()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
tensorflow      INFO     Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
VAE-BLPC0-ENCODER_compressed_256v46
Model VAE-BLPC0-ENCODER_compressed_256v46: \(\alpha :   1\),  \(\beta :   1\),  \(\gamma :   0\), latent_dim:  8, dense:512 
Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 16, 256, 1)] 0                                            
__________________________

tensorflow      INFO     Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
tensorflow      INFO     Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
tensorflow      INFO     Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
tensorflow      INFO     Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/d

In [10]:
def plot_model(history, name, i):
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2,figsize=(15,10))

    fig.suptitle("Model v%3d: alpha:%3d, beta:%3d, gamma:%3d, latent_dim:%3d, dense:%3d " % (model_num, alpha, beta, gamma, latent_dim, dens_lay))

    ax1.plot(history.history['false_loss'])
    ax1.plot(history.history['true_loss'])
    ax1.plot(history.history['val_false_loss'])
    ax1.plot(history.history['val_true_loss'])
    ax1.set_title('Model Clustering Loss')
    ax1.set_ylabel('loss')
    ax1.set_xlabel('epoch')
    ax1.legend(['false_loss', 'true_loss','val_false_loss','val_true_loss'], loc='upper right')
    ax1.grid()

    ax2.plot(history.history['false_score'])
    ax2.plot(history.history['true_score'])
    ax2.plot(history.history['val_false_score'])
    ax2.plot(history.history['val_true_score'])
    ax2.set_title('Model Clustering Score')
    ax2.set_ylabel('loss')
    ax2.set_xlabel('epoch')
    ax2.legend(['false_score', 'true_score','val_false_score','val_true_score'], loc='upper right')
    ax2.grid()

    ax3.plot(history.history['reconstruction_loss'])
    ax3.plot(history.history['val_reconstruction_loss'])
    ax3.set_title('Model Reconstruction')
    ax3.set_ylabel('loss')
    ax3.set_xlabel('epoch')
    ax3.legend(['reconstruction_loss', 'val_reconstruction_loss'], loc='upper right')
    ax3.grid()

    ax4.plot(history.history['kl_loss'])
    ax4.plot(history.history['val_kl_loss'])
    ax4.set_title('Model Divergence')
    ax4.set_ylabel('loss')
    ax4.set_xlabel('epoch')
    ax4.legend(['kl_loss', 'val_kl_loss'], loc='upper right')
    ax4.grid()
    fig.savefig(name+"-"+str(i)+'.png')

In [11]:
import time 
BATCH= 6
NUM_SAMPLES = 6000
epoch=[140,140,140,140,140,140]
for i in range(BATCH):
    print(i)
    start = time.time()
    data, false_data, true_data = create_data_set_multicore(NUM_SAMPLES=NUM_SAMPLES, snr_base=20, snr_range = 10, factor=1)
    data_test, false_data_test, true_data_test =  create_data_set_multicore(NUM_SAMPLES=1000, snr_base=20, snr_range = 10, factor=1)
    
    least = min([data.shape[0], false_data.shape[0], true_data.shape[0]])
    least_test = min([data_test.shape[0], false_data_test.shape[0], true_data_test.shape[0]])
    data, false_data, true_data  = data[:least], false_data[:least], true_data[:least]
    data_140test, false_data_test, true_data_test = data_test[:least_test], false_data_test[:least_test], true_data_test[:least_test]

    print(time.time()-start)
    data = data[:]
    history = model.fit(x =[data, true_data, false_data], y= data[:,:,:,:], epochs=epoch[i], batch_size=2000, 
          validation_data=([data_test, true_data_test, false_data_test ], data_test),validation_batch_size=6000)
    
    del data, false_data, true_data
    gc.collect()
    
    model.encoder.save(name+"-"+str(i)+".h5")
    plot_model(history, name, i)

0
Creating True
6000
(5967, 6, 16, 2048)
(5967, 6, 16, 2048)
(5967, 6, 16, 2048) (5967, 6, 16, 256)
(5967, 6, 16, 256)
(35802, 16, 256, 1)
Creating False
(35997, 6, 16, 2048)
(35997, 6, 16, 2048) (35997, 6, 16, 256)
(35997, 6, 16, 256)
Creating True
(17979, 6, 16, 2048)
(17979, 6, 16, 2048) (17979, 6, 16, 256)
(17979, 6, 16, 256)
(17979, 6, 16, 2048)
(17979, 6, 16, 2048) (17979, 6, 16, 256)
(17979, 6, 16, 256)
(35958, 6, 16, 256, 1)
Creating True
1000
(975, 6, 16, 2048)
(975, 6, 16, 2048)
(975, 6, 16, 2048) (975, 6, 16, 256)
(975, 6, 16, 256)
(5850, 16, 256, 1)
Creating False
(5967, 6, 16, 2048)
(5967, 6, 16, 2048) (5967, 6, 16, 256)
(5967, 6, 16, 256)
Creating True
(2964, 6, 16, 2048)
(2964, 6, 16, 2048) (2964, 6, 16, 256)
(2964, 6, 16, 256)
(2964, 6, 16, 2048)
(2964, 6, 16, 2048) (2964, 6, 16, 256)
(2964, 6, 16, 256)
(5928, 6, 16, 256, 1)
1019.8532917499542
Epoch 1/140
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/

Epoch 5/140
18/18 [==============================] - 15s 813ms/step - loss: 295.2116 - reconstruction_loss: 0.6877 - kl_loss: 9.9163 - false_loss: 13.0820 - true_loss: 17.7771 - false_score: 0.0000e+00 - true_score: 0.0000e+00 - val_loss: 299.4014 - val_reconstruction_loss: 0.6876 - val_kl_loss: 10.5320 - val_false_loss: 10.0178 - val_true_loss: 17.7472 - val_false_score: 0.0000e+00 - val_true_score: 0.0000e+00
Epoch 6/140
18/18 [==============================] - 14s 801ms/step - loss: 295.1808 - reconstruction_loss: 0.6877 - kl_loss: 9.9141 - false_loss: 12.5932 - true_loss: 17.6628 - false_score: 0.0000e+00 - true_score: 0.0000e+00 - val_loss: 298.8515 - val_reconstruction_loss: 0.6875 - val_kl_loss: 10.4296 - val_false_loss: 10.0823 - val_true_loss: 17.6482 - val_false_score: 0.0000e+00 - val_true_score: 0.0000e+00
Epoch 7/140
18/18 [==============================] - 14s 798ms/step - loss: 295.0508 - reconstruction_loss: 0.6877 - kl_loss: 9.9006 - false_loss: 12.2557 - true_loss: 17

Epoch 25/140
18/18 [==============================] - 12s 687ms/step - loss: 295.1066 - reconstruction_loss: 0.6877 - kl_loss: 9.9086 - false_loss: 11.0147 - true_loss: 17.3994 - false_score: 0.0000e+00 - true_score: 0.0000e+00 - val_loss: 300.6509 - val_reconstruction_loss: 0.6875 - val_kl_loss: 10.0042 - val_false_loss: 10.9211 - val_true_loss: 17.0744 - val_false_score: 0.0000e+00 - val_true_score: 0.0000e+00
Epoch 26/140
18/18 [==============================] - 12s 656ms/step - loss: 295.2238 - reconstruction_loss: 0.6877 - kl_loss: 9.9036 - false_loss: 10.9902 - true_loss: 17.3898 - false_score: 0.0000e+00 - true_score: 0.0000e+00 - val_loss: 300.4599 - val_reconstruction_loss: 0.6875 - val_kl_loss: 10.0004 - val_false_loss: 10.9008 - val_true_loss: 17.0763 - val_false_score: 0.0000e+00 - val_true_score: 0.0000e+00
Epoch 27/140
18/18 [==============================] - 13s 703ms/step - loss: 295.2885 - reconstruction_loss: 0.6877 - kl_loss: 9.9177 - false_loss: 10.9675 - true_loss:

Epoch 45/140
18/18 [==============================] - 11s 634ms/step - loss: 295.3006 - reconstruction_loss: 0.6877 - kl_loss: 9.9145 - false_loss: 10.7334 - true_loss: 17.2880 - false_score: 0.0000e+00 - true_score: 0.0000e+00 - val_loss: 298.5435 - val_reconstruction_loss: 0.6875 - val_kl_loss: 9.9679 - val_false_loss: 10.6792 - val_true_loss: 17.1128 - val_false_score: 0.0000e+00 - val_true_score: 0.0000e+00
Epoch 46/140
18/18 [==============================] - 11s 637ms/step - loss: 295.2763 - reconstruction_loss: 0.6877 - kl_loss: 9.9129 - false_loss: 10.7258 - true_loss: 17.2850 - false_score: 0.0000e+00 - true_score: 0.0000e+00 - val_loss: 298.4820 - val_reconstruction_loss: 0.6875 - val_kl_loss: 9.9666 - val_false_loss: 10.6733 - val_true_loss: 17.1128 - val_false_score: 0.0000e+00 - val_true_score: 0.0000e+00
Epoch 47/140
18/18 [==============================] - 11s 634ms/step - loss: 295.3101 - reconstruction_loss: 0.6877 - kl_loss: 9.9172 - false_loss: 10.7187 - true_loss: 1

Epoch 65/140
18/18 [==============================] - 11s 625ms/step - loss: 295.3195 - reconstruction_loss: 0.6877 - kl_loss: 9.9205 - false_loss: 10.6273 - true_loss: 17.2460 - false_score: 0.0000e+00 - true_score: 0.0000e+00 - val_loss: 297.7378 - val_reconstruction_loss: 0.6875 - val_kl_loss: 9.9512 - val_false_loss: 10.5913 - val_true_loss: 17.1235 - val_false_score: 0.0000e+00 - val_true_score: 0.0000e+00
Epoch 66/140
18/18 [==============================] - 11s 619ms/step - loss: 295.2498 - reconstruction_loss: 0.6877 - kl_loss: 9.9081 - false_loss: 10.6236 - true_loss: 17.2445 - false_score: 0.0000e+00 - true_score: 0.0000e+00 - val_loss: 297.7125 - val_reconstruction_loss: 0.6875 - val_kl_loss: 9.9497 - val_false_loss: 10.5892 - val_true_loss: 17.1233 - val_false_score: 0.0000e+00 - val_true_score: 0.0000e+00
Epoch 67/140
18/18 [==============================] - 11s 632ms/step - loss: 295.3074 - reconstruction_loss: 0.6877 - kl_loss: 9.9150 - false_loss: 10.6202 - true_loss: 1

...

Epoch 54/140
18/18 [==============================] - 11s 612ms/step - loss: 295.2499 - reconstruction_loss: 0.6877 - kl_loss: 9.9135 - false_loss: 10.4112 - true_loss: 17.1600 - false_score: 0.0000e+00 - true_score: 0.0000e+00 - val_loss: 296.0769 - val_reconstruction_loss: 0.6876 - val_kl_loss: 9.9166 - val_false_loss: 10.4088 - val_true_loss: 17.1468 - val_false_score: 0.0000e+00 - val_true_score: 0.0000e+00
Epoch 55/140
18/18 [==============================] - 11s 630ms/step - loss: 295.4493 - reconstruction_loss: 0.6877 - kl_loss: 9.9057 - false_loss: 10.4112 - true_loss: 17.1600 - false_score: 0.0000e+00 - true_score: 0.0000e+00 - val_loss: 296.0769 - val_reconstruction_loss: 0.6876 - val_kl_loss: 9.9166 - val_false_loss: 10.4087 - val_true_loss: 17.1469 - val_false_score: 0.0000e+00 - val_true_score: 0.0000e+00
Epoch 56/140
18/18 [==============================] - 11s 626ms/step - loss: 295.1389 - reconstruction_loss: 0.6877 - kl_loss: 9.9153 - false_loss: 10.4111 - true_loss: 1

Epoch 74/140
18/18 [==============================] - 11s 620ms/step - loss: 295.2644 - reconstruction_loss: 0.6877 - kl_loss: 9.9122 - false_loss: 10.4106 - true_loss: 17.1598 - false_score: 0.0000e+00 - true_score: 0.0000e+00 - val_loss: 296.0742 - val_reconstruction_loss: 0.6876 - val_kl_loss: 9.9166 - val_false_loss: 10.4081 - val_true_loss: 17.1473 - val_false_score: 0.0000e+00 - val_true_score: 0.0000e+00
Epoch 75/140
18/18 [==============================] - 11s 614ms/step - loss: 295.0746 - reconstruction_loss: 0.6877 - kl_loss: 9.9229 - false_loss: 10.4106 - true_loss: 17.1598 - false_score: 0.0000e+00 - true_score: 0.0000e+00 - val_loss: 296.0739 - val_reconstruction_loss: 0.6876 - val_kl_loss: 9.9165 - val_false_loss: 10.4081 - val_true_loss: 17.1472 - val_false_score: 0.0000e+00 - val_true_score: 0.0000e+00
Epoch 76/140
18/18 [==============================] - 11s 618ms/step - loss: 295.1654 - reconstruction_loss: 0.6877 - kl_loss: 9.9079 - false_loss: 10.4105 - true_loss: 1

Epoch 94/140
18/18 [==============================] - 11s 640ms/step - loss: 295.1078 - reconstruction_loss: 0.6877 - kl_loss: 9.9132 - false_loss: 10.4100 - true_loss: 17.1596 - false_score: 0.0000e+00 - true_score: 0.0000e+00 - val_loss: 296.0716 - val_reconstruction_loss: 0.6876 - val_kl_loss: 9.9162 - val_false_loss: 10.4081 - val_true_loss: 17.1471 - val_false_score: 0.0000e+00 - val_true_score: 0.0000e+00
Epoch 95/140
18/18 [==============================] - 11s 631ms/step - loss: 295.3833 - reconstruction_loss: 0.6877 - kl_loss: 9.9045 - false_loss: 10.4100 - true_loss: 17.1596 - false_score: 0.0000e+00 - true_score: 0.0000e+00 - val_loss: 296.0708 - val_reconstruction_loss: 0.6876 - val_kl_loss: 9.9163 - val_false_loss: 10.4079 - val_true_loss: 17.1472 - val_false_score: 0.0000e+00 - val_true_score: 0.0000e+00
Epoch 96/140
18/18 [==============================] - 11s 624ms/step - loss: 295.1110 - reconstruction_loss: 0.6877 - kl_loss: 9.9213 - false_loss: 10.4100 - true_loss: 1

Epoch 114/140
18/18 [==============================] - 11s 636ms/step - loss: 295.1388 - reconstruction_loss: 0.6877 - kl_loss: 9.9109 - false_loss: 10.4095 - true_loss: 17.1593 - false_score: 0.0000e+00 - true_score: 0.0000e+00 - val_loss: 296.0667 - val_reconstruction_loss: 0.6876 - val_kl_loss: 9.9160 - val_false_loss: 10.4076 - val_true_loss: 17.1471 - val_false_score: 0.0000e+00 - val_true_score: 0.0000e+00
Epoch 115/140
18/18 [==============================] - 11s 614ms/step - loss: 295.1379 - reconstruction_loss: 0.6877 - kl_loss: 9.9147 - false_loss: 10.4095 - true_loss: 17.1593 - false_score: 0.0000e+00 - true_score: 0.0000e+00 - val_loss: 296.0663 - val_reconstruction_loss: 0.6876 - val_kl_loss: 9.9159 - val_false_loss: 10.4077 - val_true_loss: 17.1470 - val_false_score: 0.0000e+00 - val_true_score: 0.0000e+00
Epoch 116/140
18/18 [==============================] - 11s 636ms/step - loss: 295.3916 - reconstruction_loss: 0.6877 - kl_loss: 9.9062 - false_loss: 10.4095 - true_loss

Epoch 134/140
18/18 [==============================] - 11s 617ms/step - loss: 295.1429 - reconstruction_loss: 0.6877 - kl_loss: 9.9126 - false_loss: 10.4091 - true_loss: 17.1592 - false_score: 0.0000e+00 - true_score: 0.0000e+00 - val_loss: 296.0615 - val_reconstruction_loss: 0.6876 - val_kl_loss: 9.9158 - val_false_loss: 10.4072 - val_true_loss: 17.1470 - val_false_score: 0.0000e+00 - val_true_score: 0.0000e+00
Epoch 135/140
18/18 [==============================] - 11s 625ms/step - loss: 295.2207 - reconstruction_loss: 0.6877 - kl_loss: 9.9115 - false_loss: 10.4091 - true_loss: 17.1592 - false_score: 0.0000e+00 - true_score: 0.0000e+00 - val_loss: 296.0617 - val_reconstruction_loss: 0.6876 - val_kl_loss: 9.9159 - val_false_loss: 10.4072 - val_true_loss: 17.1470 - val_false_score: 0.0000e+00 - val_true_score: 0.0000e+00
Epoch 136/140
18/18 [==============================] - 11s 616ms/step - loss: 295.4113 - reconstruction_loss: 0.6877 - kl_loss: 9.9122 - false_loss: 10.4090 - true_loss

In [12]:
model.encoder.save(name+".h5")

In [13]:
%matplotlib inline
import random 
for i in range(10):
    index = int(random.random()*1000)
    plot_data = data[index:index+1,:,:]
    plt.figure(figsize=(10,4))
    plt.xlabel("Fchans")
    plt.ylabel("Time")
    plt.imshow(plot_data[0,:,:,0], interpolation='nearest', cmap=plt.get_cmap('hot'))
    plt.show()

    stuff = model.encoder.predict(plot_data)
    sample = sample_creation(stuff)
    reconstruction = model.decoder.predict(sample)
    plt.figure(figsize=(10,4))
    plt.title("reconstruction")
    plt.xlabel("Fchans")
    plt.ylabel("Time")
    plt.imshow(reconstruction[0,:,:,0], interpolation='nearest', cmap=plt.get_cmap('hot'))


NameError: name 'data' is not defined